In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from glob import glob
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from scipy import stats

In [2]:
mu_b = 5.7883818012 * 10**(-5)
mu_b

5.7883818012e-05

In [3]:
def last_23(list_arg):
    return(list_arg[-23:])

In [4]:
list_sweepVT = sorted(glob('*Voltage_Sweep.tsv.gz'))

In [5]:
sweepVT_sp = sorted(list_sweepVT[6:], key=last_23)
sweepVT_sm = sorted(list_sweepVT[:6], key=last_23)

In [6]:
sorted(list_sweepVT[6:], key=last_23)

['02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_67deg_-3T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_74,6deg_-4T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_82,2deg_-5T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_89,8deg_-6T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_97,4deg_-7T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmap-det_105deg_-8T.Voltage_Sweep.tsv.gz']

In [25]:
sorted(list_sweepVT[:6], key=last_23)

['02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_-23deg_-3T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_-15,4deg_-4T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_-7,8deg_-5T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_-0,2deg_-6T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_7,4deg_-7T.Voltage_Sweep.tsv.gz',
 '02_WSe2-671nm-4uW-10-+30V-20s_lin-exc_sigmam-det_15deg_-8T.Voltage_Sweep.tsv.gz']

In [7]:
list_calibration = glob('*.wlen_to_px.tsv.gz')

In [8]:
calibration = 1230.8 / (pd.read_csv(list_calibration[0], header=None).as_matrix().transpose())[0]

In [9]:
sweepVT_sigmam = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in sweepVT_sp])
sweepVT_sigmap = np.array([pd.read_csv(file, sep='\t', header=None).as_matrix() for file in sweepVT_sm])

In [10]:
np.shape(sweepVT_sigmam)


(6, 81, 1340)

In [11]:
np.shape(sweepVT_sigmap)

(6, 81, 1340)

In [12]:
np.shape(sweepVT_sigmap[0,0,:])

(1340,)

In [13]:
spec = sweepVT_sigmap[0,15,:]
calibration

array([1.85623947, 1.85587281, 1.85550909, ..., 1.47222415, 1.47199701,
       1.47177171])

In [14]:
output_notebook()

Loading BokehJS ...

In [236]:
def lorentz(x, x_n, G):
    X = (x_n - x)/(G/2)
    return 1/(1+X**2)
def lorentz_double(x, xn1, G1, A1, xn2, G2, A2):
    return(lorentz(x, xn1, G1, A1) + lorentz(x, xn2, G2, A2))

In [16]:
a_min, a_max = 1.69, 1.72

In [237]:
def fit_a(data,
          cal=calibration,
          a_min=1.695,
          a_max=1.72,
          base_function=lorentz,
          p0=[1.70, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_t(data,
          cal=calibration,
          a_min=1.66,
          a_max=1.69,
          base_function=lorentz,
          p0=[1.67, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_td(data,
          cal=calibration,
          a_min=1.638,
          a_max=1.654,
          base_function=lorentz,
          p0=[1.645, 0.05]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(lorentz, cal_slice, data_slice, p0=p0)
    return popt[0]
def fit_Tr(data,
          cal=calibration,
          a_min=1.62,#1.66,
          a_max=1.69,#1.69,
          base_function=lorentz_double,
          p0=[1.63, 0.05, 10, 1.67, 0.05, 10]):
    data_slice = data[np.logical_and(cal>a_min, cal<a_max)]
    #data_slice = data_slice/data_slice.max()
    cal_slice = cal[np.logical_and(cal>a_min, cal<a_max)]
    popt, pcov = curve_fit(base_function, cal_slice, data_slice, p0=p0)
    return popt


In [232]:
y = sweepVT_sigmam[0,9,:]
Fit = fit_Tr(y)
a, b, c, d, e, f = Fit[0], Fit[1], Fit[2], Fit[3], Fit[4], Fit[5]
y2 = lorentz_double(calibration, a, b, c, d, e, f)
y = y/y.max()
pp = figure()
pp.line(calibration, y)
pp.line(calibration, y2, color='red')
show(pp)

In [243]:
a_energies_sp = np.array([np.mean([fit_a(sweepVT_sigmap[j,i,:]) for i in np.arange(18,27,1)]) for j in range(0,6)])
a_energies_sm = np.array([np.mean([fit_a(sweepVT_sigmam[j,i,:]) for i in np.arange(18,27,1)]) for j in range(0,6)])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
a_energies_sp

array([1.70576786, 1.70574761, 1.70569657, 1.70561589, 1.70544315,
       1.70521966])

In [160]:
p = figure(y_range=(1.705, 1.709))
xxx = np.arange(0,6,1)
p.scatter(xxx, a_energies_sm)
p.scatter(xxx, a_energies_sp, color='red')
show(p)

In [170]:
#a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[:,18,:]])
#a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[:,18,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(-3, -9, -1))
#a_energies_sp
a_energies_sp = np.array([fit_a(field) for field in sweepVT_sigmap[0,:,:]])
a_energies_sm = np.array([fit_a(field) for field in sweepVT_sigmam[0,:,:]])
#g_factors = (a_energies_sm - a_energies_sp)/(mu_b * np.arange(9))
#a_energies_sp
g_factors

/usr/lib/python3/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


array([-1.70674208, -2.28752658, -2.96550302, -3.30096911, -3.2287998 ,
       -3.6816929 ])

In [171]:
p = figure(y_range=(1.705, 1.71))
xxx = np.arange(0,81,1)
p.scatter(xxx, a_energies_sm)
p.scatter(xxx, a_energies_sp, color='red')
show(p)

In [241]:
p = figure()
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(-3, -9, -1) * mu_b, a_energies_sm)
teslas = np.arange(-3, -9, -1) * mu_b
linear_sm = slope * teslas + intercept
p.scatter(teslas, a_energies_sm)
p.line(mu_b * np.arange(-3, -9, -1), linear_sm, color='red')
slope_sm = slope
offset_sm = intercept
err_sm = std_err

slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(-3, -9, -1) * mu_b, a_energies_sp)
teslas = np.arange(-3, -9, -1) * mu_b
linear_sp = slope * teslas + intercept
p.scatter(teslas, a_energies_sp)
p.line(mu_b * np.arange(-3, -9, -1), linear_sp, color='green')
slope_sp = slope
offset_sp = intercept
err_sp = std_err

g_factor = (slope_sp - slope_sm) + (offset_sp - offset_sm)
print((offset_sp - offset_sm))
print(g_factor)
print(np.sqrt(err_sm**2 + err_sp**2))
#plt.plot(calibration, y)
show(p)

0.0015304861456348906
25.138714212995822
10.293726656418013


In [ ]:
sweepVT_sigmap[:,15,:]

In [ ]:
p = figure()
p.line(calibration, spec)
show(p)